<a href="https://colab.research.google.com/github/ashwinsapre/GANfaces_iCarl/blob/main/iCarl_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [105]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow import keras
import keras.layers as L
from keras.datasets import fashion_mnist
from keras.applications import DenseNet121
from keras.applications import VGG16
import matplotlib.pyplot as plt

 For multi-task learning, branching NN out
- one branch predicts the class of FashionMNIST object (classification)
- other branch predicts whether the object is a "top" or not (0/1 output)

In [106]:
def calc_feature_vectors(model, images):
    #extract feature vector from a layer of ALREADY-TRAINED model
    #size of feature vector=784
    feature_model=keras.Model(inputs=model.inputs, outputs=model.get_layer("flatten").output)
    feature_vectors=feature_model.predict(images)
    norms=np.linalg.norm(feature_vectors, keepdims=True, axis=1)
    feature_vectors=feature_vectors/norms
    return feature_vectors

In [107]:
def create_exemplar_set(mem_size, n_classes, feature_vectors, labels, reconstruct=False):
    per_class=mem_size//n_classes

    class_vectors={}
    mean_class_vectors={}
    class_vectors_distances={}

    #init dicts
    for i in range(n_classes):
        class_vectors[i]=[]
        mean_class_vectors[i]=[]
        class_vectors_distances[i]=[]
    
    #vectors belonging to class i go in the list in key i of class_vectors
    for i in range(len(labels)):
        class_vectors[labels[i]].append(feature_vectors[i])

    #calculate mean class vectors by summing all vectors per class and diving by length
    for i in range(n_classes):
        mean_class_vectors[i]=np.sum(class_vectors[i], axis=0)/len(class_vectors[i])
    
    #calculating distances from mean
    for i in range(len(labels)):
        class_vectors_distances[labels[i]].append(np.linalg.norm(mean_class_vectors[labels[i]] - feature_vectors[i]))

    #sorting vectors by their corresponding distances from class means
    for i in range(n_classes):
        class_vectors_distances[i], class_vectors[i] = (list(t) for t in zip(*sorted(zip(class_vectors_distances[i], class_vectors[i]))))

    exemplars_x = []
    exemplars_y = []

    #choose 'per_class' number of vectors
    for i in range(n_classes):
        exemplars_x.append(class_vectors[i][:per_class])
        exemplars_y+= per_class*[i]

    return exemplars_x, exemplars_y

In [108]:
def classify(X, mean_class_vectors, n_classes):
    #image embedding created
    image_vector=calc_feature_vectors(model, X)
    #distance to all mean vectors calculated
    distances=[]
    indices=[x for x in range(n_classes)]
    #sort class values by their corresponding distances
    for i in range(n_classes):
        distances.append(np.linalg.norm(mean_class_vectors[i] - image_vector))
    distances, indices = (list(t) for t in zip(*sorted(zip(distances, indices))))
    #return the class with the least distance
    return indices[0]

In [109]:
def create_model(n_classes, input_dim, cl_weight, b_weight, lr):
    '''
        Creating categorical classification model
    '''
    
    inputs=L.Input((input_dim,input_dim,1), name='input_layer_common')
    
    xc=L.Conv2D(64, kernel_size=3, padding='same', strides=1, name='conv1_c')(inputs)
    xc=L.LeakyReLU(0.2, name='relu1_c')(xc)
    xc=L.Conv2D(64, kernel_size=3, padding='same', strides=1, name='conv2_c')(xc)
    xc=L.LeakyReLU(0.2, name='relu2_c')(xc)
    xc=L.MaxPool2D(pool_size=2, strides=2, name='pool1_c')(xc)
    
    xc=L.Conv2D(32, kernel_size=3, padding='same', strides=1, name='conv3_c')(xc)
    xc=L.LeakyReLU(0.2, name='relu3_c')(xc)
    xc=L.Conv2D(32, kernel_size=3, padding='same', strides=1, name='conv4_c')(xc)
    xc=L.LeakyReLU(0.2, name='relu4_c')(xc)
    xc=L.MaxPool2D(pool_size=2, strides=2, name='pool2_c')(xc)
    
    xc=L.Conv2D(16, kernel_size=3, padding='same', strides=1, name='conv5_c')(xc)
    xc=L.LeakyReLU(0.2, name='relu5_c')(xc)
    xc=L.Conv2D(16, kernel_size=3, padding='same', strides=1, name='conv6_c')(xc)
    xc=L.LeakyReLU(0.2, name='relu6_c')(xc)
    xc=L.MaxPool2D(pool_size=2, strides=2, name='pool3_c')(xc)
    
    xc=L.Flatten(name='flatten_c')(xc)
    outputc=L.Dense(n_classes, activation='softmax', name='outputc')(xc)
    #outputc=L.Softmax(name='outputc')(xc)
    
    '''
        Creating binary classification model (a top/not a top)
    '''
    x=L.Conv2D(32, kernel_size=3, padding='same', strides=1, name='conv1_b')(inputs)
    x=L.LeakyReLU(0.2, name='relu1_b')(x)
    x=L.Conv2D(32, kernel_size=3, padding='same', strides=1, name='conv2_b')(x)
    x=L.LeakyReLU(0.2, name='relu2_b')(x)
    x=L.MaxPool2D(pool_size=2, strides=2, name='pool1_b')(x)
    
    x=L.Conv2D(16, kernel_size=3, padding='same', strides=1, name='conv3_b')(x)
    x=L.LeakyReLU(0.2, name='relu3_b')(x)
    x=L.Conv2D(16, kernel_size=3, padding='same', strides=1, name='conv4_b')(x)
    x=L.LeakyReLU(0.2, name='relu4_b')(x)
    x=L.MaxPool2D(pool_size=2, strides=2, name='pool2_b')(x)
    
    x=L.Flatten(name='flatten_b')(x)
    outputb=L.Dense(1, activation='sigmoid', name='outputb')(x)
    #outputb=L.Activation('sigmoid', name="outputb")(x)

    '''
        Combining both models
    '''
    model=keras.Model(inputs=inputs, outputs=[outputc, outputb])
    losses={'outputc':'categorical_crossentropy', 'outputb':'binary_crossentropy'}
    loss_weights=[cl_weight, b_weight]
    model.compile(optimizer=keras.optimizers.Adam(lr), 
                      loss=losses,
                      loss_weights=loss_weights,
                      metrics=['accuracy'])

    return model

In [110]:
def fine_tune(model, n_classes, cl_weight, b_weight, lr):
    penultimate_layer=model.get_layer('flatten_c').output
    outputc=L.Dense(n_classes, activation='softmax', name='outputc')(penultimate_layer)

    outputb=model.get_layer('outputb').output
    new_model=keras.Model(inputs=model.inputs,
                        outputs=[outputc, outputb])
    for layer in new_model.layers:
        if(layer.name not in ['conv4_b', 'outputb', 'conv6_c', 'outputc']):
            layer.trainable=False
    losses={'outputc':'categorical_crossentropy', 'outputb':'binary_crossentropy'}
    loss_weights=[cl_weight, b_weight]
    new_model.compile(optimizer=keras.optimizers.Adam(lr), loss=losses, loss_weights=loss_weights, metrics=['accuracy'])

    return new_model

In [111]:
def train_model(model, n_epochs:int, x_train, y_trainc, y_trainb, validation_split):
    history=model.fit(x=x_train, 
                      y={'outputc':y_trainc, 'outputb': y_trainb}, 
                      epochs=n_epochs, 
                      validation_split=0.1,
                      shuffle=True,
                      verbose=verbose)
    return model

In [112]:
def synthesize_dataset(x_train, y_train, n):
    labeldict={}
    labeldict[0]=1
    labeldict[1]=0
    labeldict[2]=1
    labeldict[3]=1
    labeldict[4]=1
    labeldict[5]=0
    labeldict[6]=1
    labeldict[7]=0
    labeldict[8]=0
    labeldict[9]=0

    classes=[x for x in range(n)]
    selected_indices=[]
    unselected_indices=[]
    for i in range(len(y_train)):
        if(y_train[i] in classes):
            selected_indices.append(i)
        else:
            unselected_indices.append(i)

    x_train_new=np.delete(x_train, [unselected_indices], axis=0)
    y_train=[y_train[i] for i in selected_indices]

    y_trainb=[]
    y_testb=[]
    y_trainc=keras.utils.to_categorical(y_train)
    #y_testc=keras.utils.to_categorical(y_test)
    
    for i in range(len(y_train)):
        y_trainb.append(labeldict[y_train[i]])
        
    #for i in range(len(y_test)):
    #    y_testb.append(labeldict[y_test[i]])
    
    return x_train_new, np.asarray(y_trainc), np.asarray(y_trainb)

In [118]:
lr=0.0005
epochs=5
init_classes=3
verbose=1

In [119]:
(x_train, y_train), (x_test, y_test)=tf.keras.datasets.fashion_mnist.load_data()
x_train=x_train.reshape((x_train.shape[0], 28,28,1))
x_test=x_test.reshape((x_test.shape[0],28,28,1))
x_train=x_train/255.0
x_test=x_test/255.0

#first iteration of training with 5 classes
xt, ytc, ytb=synthesize_dataset(x_train, y_train, init_classes)
model=create_model(n_classes=init_classes, input_dim=28, cl_weight=1.0, b_weight=1.5, lr=lr)
train_model(model, epochs, xt, ytc, ytb, validation_split=0.1)

Epoch 1/5
507/507 [==============================] - 10s 18ms/step - loss: 0.5660 - outputc_loss: 0.2909 - outputb_loss: 0.1834 - outputc_accuracy: 0.8954 - outputb_accuracy: 0.9064 - val_loss: 0.1314 - val_outputc_loss: 0.0809 - val_outputb_loss: 0.0337 - val_outputc_accuracy: 0.9689 - val_outputb_accuracy: 0.9900
Epoch 2/5
507/507 [==============================] - 9s 17ms/step - loss: 0.1082 - outputc_loss: 0.0766 - outputb_loss: 0.0210 - outputc_accuracy: 0.9751 - outputb_accuracy: 0.9941 - val_loss: 0.1087 - val_outputc_loss: 0.0775 - val_outputb_loss: 0.0208 - val_outputc_accuracy: 0.9750 - val_outputb_accuracy: 0.9939
Epoch 3/5
507/507 [==============================] - 8s 17ms/step - loss: 0.1009 - outputc_loss: 0.0725 - outputb_loss: 0.0190 - outputc_accuracy: 0.9767 - outputb_accuracy: 0.9934 - val_loss: 0.1069 - val_outputc_loss: 0.0699 - val_outputb_loss: 0.0246 - val_outputc_accuracy: 0.9744 - val_outputb_accuracy: 0.9917
Epoch 4/5
507/507 [==============================] 

In [120]:
#second iteration of training with 9 classes
xt, ytc, ytb=synthesize_dataset(x_train, y_train, init_classes+7)
ftmodel=fine_tune(model, n_classes=init_classes+7, cl_weight=1, b_weight=1, lr=0.001)
train_model(ftmodel, epochs, xt, ytc, ytb, validation_split=0.1)

Epoch 1/5
1688/1688 [==============================] - 19s 11ms/step - loss: 0.8941 - outputc_loss: 0.8030 - outputb_loss: 0.0911 - outputc_accuracy: 0.7280 - outputb_accuracy: 0.9684 - val_loss: 0.4324 - val_outputc_loss: 0.3990 - val_outputb_loss: 0.0334 - val_outputc_accuracy: 0.8480 - val_outputb_accuracy: 0.9898
Epoch 2/5
1688/1688 [==============================] - 18s 11ms/step - loss: 0.4138 - outputc_loss: 0.3795 - outputb_loss: 0.0344 - outputc_accuracy: 0.8583 - outputb_accuracy: 0.9899 - val_loss: 0.3895 - val_outputc_loss: 0.3633 - val_outputb_loss: 0.0262 - val_outputc_accuracy: 0.8663 - val_outputb_accuracy: 0.9910
Epoch 3/5
1688/1688 [==============================] - 18s 10ms/step - loss: 0.3792 - outputc_loss: 0.3521 - outputb_loss: 0.0271 - outputc_accuracy: 0.8691 - outputb_accuracy: 0.9918 - val_loss: 0.3935 - val_outputc_loss: 0.3657 - val_outputb_loss: 0.0278 - val_outputc_accuracy: 0.8648 - val_outputb_accuracy: 0.9922
Epoch 4/5
1688/1688 [======================